In [ ]:
import pymedx_custom
import logging

from pymedx_custom import PubMed
from pymedx_custom.helpers import pretty_print_xml, getContent
import logging
logging.basicConfig(level=logging.DEBUG,datefmt='%H:%M:%S' , format='%(asctime)s.%(msecs)03d - %(module)s/%(funcName)s:%(lineno)d - %(levelname)s - %(message)s')

In [11]:
pubmed = PubMed()

In [ ]:
citin_articles = pubmed.getCitingArticles('28444919')

In [20]:
article = citin_articles[0]
article.abstract

'The synthesis of bimetallic iron-nickel nanoparticles with control over the synthesized phases, particle size, surface chemistry, and oxidation level remains a challenge that limits the application of these nanoparticles. Pulsed laser ablation in liquid allows the properties tuning of the generated nanoparticles by changing the ablation solvent. Organic solvents such as acetone can minimize nanoparticle oxidation. Yet, economical laboratory and technical grade solvents that allow cost-effective production of FeNi nanoparticles contain water impurities, which are a potential source of oxidation. Here, we investigated the influence of water impurities in acetone on the properties of FeNi nanoparticles generated by pulsed laser ablation in liquids. To remove water impurities and produce "dried acetone", cost-effective and reusable molecular sieves (3 Å) are employed. The results show that the Fe50Ni50 nanoparticles\' properties are influenced by the water content of the solvent. The meta

In [13]:
pretty_print_xml(article.xml)

NameError: name 'article' is not defined

Get publications by Author

In [5]:
author = "Barcikowski"
query = author + '[Au]'
query = 'pubmed_pubmed_citedin+AND+from_uid=28444919'
articles = list(pubmed.query(query, max_results=500))
dict_articles = [x.toDict() for x in articles]
print(f"Total articles: {len(dict_articles)}")
df = pd.DataFrame(dict_articles)

12:22:39.672 - api/_get:297 - DEBUG - Attempt 1/10
12:22:39.672 - api/_get:302 - DEBUG - Requesting https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi with parameters {'tool': 'my_tool', 'email': 'my_email@example.com', 'db': 'pubmed', 'term': 'pubmed_pubmed_citedin+AND+from_uid=28444919', 'retmax': 1, 'retmode': 'json'}
12:22:39.673 - connectionpool/_new_conn:1049 - DEBUG - Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443
12:22:40.698 - connectionpool/_make_request:544 - DEBUG - https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/esearch.fcgi?tool=my_tool&email=my_email%40example.com&db=pubmed&term=pubmed_pubmed_citedin%2BAND%2Bfrom_uid%3D28444919&retmax=1&retmode=json HTTP/1.1" 200 None
12:22:40.702 - api/_get:313 - DEBUG - Response: {'header': {'type': 'esearch', 'version': '0.3'}, 'esearchresult': {'count': '0', 'retmax': '0', 'retstart': '0', 'idlist': [], 'translationset': [], 'querytranslation': '"pubmed pubmed"[All Fields] AND "uid"[All Fields]', 'erro

Total articles: 0


NameError: name 'pd' is not defined

Load current db

In [ ]:
all_adrticles = pd.read_csv('articles.csv')

merge new articles to the whole db

In [ ]:
pd.concat([all_adrticles, df],ignore_index=True).to_csv('articles1.csv', index=False)


In [ ]:
db = pd.read_csv('articles1.csv')


In [ ]:
pd.set_option('display.max_rows', None)
db.loc[db['pubmed_id'] == 28444919]['doi']

164    10.1002/cphc.201700310
Name: doi, dtype: object